In [133]:
# import
import itertools
import re
import numpy as np
import pandas as pd

In [134]:
# check steam dataset
pd.read_csv('steam_cleaned.csv').head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [135]:
# define function to remove all non-english games
def remove_non_english(df):
    # keep only rows marked as supporting english
    df = df[df['english'] == 1].copy()
    
    # keep rows which don't contain 3 or more non-ascii characters in succession
    df = df[~df['name'].str.contains('[^\u0001-\u007F]{3,}')]
    
    # remove english column, now redundant
    df = df.drop('english', axis=1)
    
    return df

In [136]:
# define function for rating score based on SteamDB method
def calc_rating(row):
    import math

    pos = row['positive_ratings']
    neg = row['negative_ratings']

    total_reviews = pos + neg
    average = pos / total_reviews
    
    # pulls score towards 50, pulls more strongly for games with few reviews
    score = average - (average*0.5) * 2**(-math.log10(total_reviews + 1))

    return score * 100

In [137]:
# define function to process categories & genres
def process_cat_gen_tag(df):
    # only going to use these categories
    cat_cols = [
        # 'Local Multi-Player',
        # 'MMO',
        # 'Mods',
        'Multi-player',
        # 'Online Co-op',
        # 'Online Multi-Player',
        'Single-player'
    ]
    
    # create a new column for each category, with 1s indicating membership and 0s for non-members
    for col in sorted(cat_cols):
        col_name = re.sub(r'[\s\-\/]', '_', col.lower())
        col_name = re.sub(r'[()]', '', col_name)
        
        df[col_name] = df['categories'].apply(lambda x: 1 if col in x.split(';') else 0)
        
    # repeat for genre column names (only keeping 'main' genres similar to steam store)
    gen_cols = [
        'Action',
        'Adventure',
        'Casual',
        'Indie',
        'Massively Multiplayer',
        'RPG',
        'Racing',
        'Simulation',
        'Sports',
        'Strategy'
    ]
    
    gen_col_names = []
    
    # create new columns for each genre with 1s for games of that genre
    for col in sorted(gen_cols):
        col_name = col.lower().replace('&', 'and').replace(' ', '_')
        gen_col_names.append(col_name)
        
        df[col_name] = df['genres'].apply(lambda x: 1 if col in x.split(';') else 0)
    
    # remove "non-games" based on genre
    gen_sums = df[gen_col_names].sum(axis=1)
    df = df[gen_sums > 0].copy()

    # remove redundant columns and return dataframe (keeping genres column for reference)
    df = df.drop(['categories', 'steamspy_tags', 'genres'], axis=1)
    
    return df

In [138]:
# define function to preprocess steam dataset for model
def pre_process():
    df = pd.read_csv('steam_cleaned.csv')
    
    # keep english only
    df = remove_non_english(df)
    
    # keep windows only, and remove platforms column
    df = df[df['platforms'].str.contains('windows')].drop('platforms', axis=1).copy()
    
    # keep lower bound of owners column, as integer
    df['owners'] = df['owners'].str.split('-').apply(lambda x: x[0]).astype(int)
    
    # calculate rating, as well as simple ratio for comparison
    df['total_ratings'] = df['positive_ratings'] + df['negative_ratings']
    df['rating_ratio'] = df['positive_ratings'] / df['total_ratings']
    df['rating'] = df.apply(calc_rating, axis=1)
    
    # convert release_date to datetime type and create separate column for release_year
    df['release_date'] = df['release_date'].astype('datetime64[ns]')
    df['release_year'] = df['release_date'].apply(lambda x: x.year)
    df['release_month'] = df['release_date'].apply(lambda x: x.month)
    
    # process genres, categories and steamspy_tag columns
    df = process_cat_gen_tag(df)
    
    return df

data = pre_process()

print('Verify no missing values:')
print(data.isnull().sum().value_counts())
data.head()

Verify no missing values:
0    30
dtype: int64


,appid,name,release_date,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,...,action,adventure,casual,indie,massively_multiplayer,rpg,racing,simulation,sports,strategy
0,10,Counter-Strike,2000-11-01,Valve,Valve,0,0,124534,3339,17612,...,1,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,0,0,3318,633,277,...,1,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,2003-05-01,Valve,Valve,0,0,3416,398,187,...,1,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,0,0,1273,267,258,...,1,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,0,0,5250,288,624,...,1,0,0,0,0,0,0,0,0,0


In [139]:
# pick top 10 developers
dev_cols = [
    'Valve',
    'id Software',
    'Ubisoft Montreal',
    'Bethesda Game Studios',
    'Daedalic Entertainment',
    'CAPCOM Co., Ltd.',
    'Klei Entertainment',
    'Square Enix',
    'CD PROJEKT RED',
    'The Behemoth'
]

# transform developer column
data.loc[~data["developer"].isin(dev_cols), "developer"] = "Other"

In [140]:
from sklearn.preprocessing import OneHotEncoder

# creating another instance of one-hot-encoder
oe_style = OneHotEncoder()

# pass publisher column
encoded_dev_df = pd.DataFrame(oe_style.fit_transform(data[["developer"]]).toarray(), columns=oe_style.categories_)

# add prefix to column names
encoded_dev_df = encoded_dev_df.add_prefix('developer_')

# check encoded publishers
encoded_dev_df.head()

,developer_Bethesda Game Studios,"developer_CAPCOM Co., Ltd.",developer_CD PROJEKT RED,developer_Daedalic Entertainment,developer_Klei Entertainment,developer_Other,developer_Square Enix,developer_The Behemoth,developer_Ubisoft Montreal,developer_Valve,developer_id Software
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [141]:
# repeat for publishers
pub_cols = [
    'Ubisoft', 
    'Valve', 
    'Devolver Digital', 
    'Square Enix', 
    'SEGA',
    'Bethesda Softworks', 
    'THQ Nordic', 
    'Paradox Interactive',
    'Electronic Arts', 
    'Sekai Project'
]

# transform
data.loc[~data["publisher"].isin(pub_cols), "publisher"] = "Other"

In [142]:
# creating another instance of one-hot-encoder
oe_style = OneHotEncoder()

# pass publisher column
encoded_pub_df = pd.DataFrame(oe_style.fit_transform(data[["publisher"]]).toarray(), columns=oe_style.categories_)

# fix column names
encoded_pub_df = encoded_pub_df.add_prefix('publisher_')

# check encoded publishers
encoded_pub_df.head()

,publisher_Bethesda Softworks,publisher_Devolver Digital,publisher_Electronic Arts,publisher_Other,publisher_Paradox Interactive,publisher_SEGA,publisher_Sekai Project,publisher_Square Enix,publisher_THQ Nordic,publisher_Ubisoft,publisher_Valve
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [143]:
# merge on key values
encoded_dev_pub_df = encoded_pub_df.join(encoded_dev_df)
encoded_dev_pub_df.head()

,publisher_Bethesda Softworks,publisher_Devolver Digital,publisher_Electronic Arts,publisher_Other,publisher_Paradox Interactive,publisher_SEGA,publisher_Sekai Project,publisher_Square Enix,publisher_THQ Nordic,publisher_Ubisoft,...,"developer_CAPCOM Co., Ltd.",developer_CD PROJEKT RED,developer_Daedalic Entertainment,developer_Klei Entertainment,developer_Other,developer_Square Enix,developer_The Behemoth,developer_Ubisoft Montreal,developer_Valve,developer_id Software
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [144]:
# merge with main df
data = data.join(encoded_dev_pub_df)

# remove redundant columns
data = data.drop(['developer', 'publisher'], axis=1)

# check df
data.head()

,appid,name,release_date,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,...,"(developer_CAPCOM Co., Ltd.,)","(developer_CD PROJEKT RED,)","(developer_Daedalic Entertainment,)","(developer_Klei Entertainment,)","(developer_Other,)","(developer_Square Enix,)","(developer_The Behemoth,)","(developer_Ubisoft Montreal,)","(developer_Valve,)","(developer_id Software,)"
0,10,Counter-Strike,2000-11-01,0,0,124534,3339,17612,317,10000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,Team Fortress Classic,1999-04-01,0,0,3318,633,277,62,5000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,30,Day of Defeat,2003-05-01,0,0,3416,398,187,34,5000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,40,Deathmatch Classic,2001-06-01,0,0,1273,267,258,184,5000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,50,Half-Life: Opposing Force,1999-11-01,0,0,5250,288,624,415,5000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [145]:
# ensure name is only object/categorical column remaining
data.dtypes

appid                                           int64
name                                           object
release_date                           datetime64[ns]
required_age                                    int64
achievements                                    int64
positive_ratings                                int64
negative_ratings                                int64
average_playtime                                int64
median_playtime                                 int64
owners                                          int64
price                                         float64
total_ratings                                   int64
rating_ratio                                  float64
rating                                        float64
release_year                                    int64
release_month                                   int64
multi_player                                    int64
single_player                                   int64
action                      

In [146]:
# export
data.to_csv('steam_preproccessed.csv')